In [11]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
materials = pd.read_csv('/Users/allenyang/Downloads/blending_materials.csv')
blends = pd.read_csv('/Users/allenyang/Downloads/blending_blends.csv')

# Define sets
# For materials, use a default index as identifier if no explicit 'Material' column is given.
material_ids = list(materials.index)  # Now using index, 0 to 99
blend_ids = list(blends.index)          # Using index, 0 to 49

# Number of raw materials and blends (expected 100 and 50)
num_materials = len(material_ids)
num_blends = len(blend_ids)

# Parameters from materials.csv (using lowercase column names)
cost = {j: materials.loc[idx, 'cost'] for idx, j in enumerate(material_ids)}
availability = {j: materials.loc[idx, 'availability'] for idx, j in enumerate(material_ids)}
pmax = {j: materials.loc[idx, 'p_max'] for idx, j in enumerate(material_ids)}

# Parameters from blends.csv (using lowercase column names)
demand = {i: blends.loc[idx, 'demand'] for idx, i in enumerate(blend_ids)}
Qmin = {i: blends.loc[idx, 'quality_min'] for idx, i in enumerate(blend_ids)}
Qmax = {i: blends.loc[idx, 'quality_max'] for idx, i in enumerate(blend_ids)}

# Identify quality contribution columns (exclude 'quality_min' and 'quality_max')
quality_cols = [col for col in blends.columns 
                if col.startswith('quality_') and col not in ['quality_min', 'quality_max']]
if len(quality_cols) != num_materials:
    raise ValueError(f"Expected {num_materials} quality contribution columns, found {len(quality_cols)}")

# Create quality parameter: quality[(i,j)] = quality contribution of material j in blend i
quality = {}
# Assuming the quality columns are sorted properly (quality_1 to quality_100)
quality_cols = sorted(quality_cols, key=lambda x: int(x.split('_')[1]))
for i in blend_ids:
    for j_idx, j in enumerate(material_ids):
        quality[(i, j)] = blends.loc[i, quality_cols[j_idx]]

# Create the optimization model
model = gp.Model("EcoClean_Blending")

# Decision variables: x[i,j] = tons of raw material j used in blend i
x = {}
for i in blend_ids:
    for j in material_ids:
        x[i, j] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")
model.update()

# Objective: Minimize total production cost
model.setObjective(gp.quicksum(cost[j] * x[i, j] for i in blend_ids for j in material_ids), GRB.MINIMIZE)

# Constraint 1: Demand satisfaction for each blend
for i in blend_ids:
    model.addConstr(gp.quicksum(x[i, j] for j in material_ids) == demand[i],
                    name=f"Demand_blend_{i}")

# Constraint 2: Quality constraints for each blend
for i in blend_ids:
    model.addConstr(gp.quicksum(quality[(i, j)] * x[i, j] for j in material_ids) >= Qmin[i] * demand[i],
                    name=f"QualityMin_blend_{i}")
    model.addConstr(gp.quicksum(quality[(i, j)] * x[i, j] for j in material_ids) <= Qmax[i] * demand[i],
                    name=f"QualityMax_blend_{i}")

# Constraint 3: Raw material proportion in each blend
for i in blend_ids:
    for j in material_ids:
        model.addConstr(x[i, j] <= pmax[j] * demand[i],
                        name=f"pmax_blend_{i}_material_{j}")

# Constraint 4: Raw material availability across all blends
for j in material_ids:
    model.addConstr(gp.quicksum(x[i, j] for i in blend_ids) <= availability[j],
                    name=f"Availability_material_{j}")

model.update()

# Optimize the model
model.optimize()

# Print the minimum production cost if an optimal solution is found
if model.status == GRB.OPTIMAL:
    print("Minimum production cost: ", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5250 rows, 5000 columns and 25000 nonzeros
Model fingerprint: 0xb545baba
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
Presolve removed 5081 rows and 0 columns
Presolve time: 0.02s
Presolved: 169 rows, 5050 columns, 13500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9775478e+04   2.316262e+04   0.000000e+00      0s
      93    3.0879621e+04   0.000000e+00   0.000000e+00      0s

Solved in 93 iterations and 0.03 seconds (0.01 work units)
Optimal objective  3.087962126e+04
Minimum production cost:  30879.621255047165


In [12]:
# --- Check original usage for raw material 5 ---
usage_original = [x[i, 5].X for i in blend_ids]
print("Original usage of raw material 5:", usage_original)
# Expect all values to be 0 (or near-zero)

# --- Reduce cost of raw material 5 by $1 ---
for i in blend_ids:
    x[i, 5].Obj -= 1  # reduce objective coefficient by 1
model.update()
model.optimize()

# --- Check new usage for raw material 5 ---
usage_new = [x[i, 5].X for i in blend_ids]
print("New usage of raw material 5:", usage_new)
# If any value > 0, material 5 is now used.


Original usage of raw material 5: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5250 rows, 5000 columns and 25000 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.9924477e+28   2.067543e+31   5.992448e-02      0s
      20    3.0798916e+04   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.03 seconds (0.00 work units

In [13]:
# Retrieve dual values (shadow prices) for the quality minimum constraints
shadow_prices = {}
for i in blend_ids:
    constr = model.getConstrByName(f"QualityMin_blend_{i}")
    shadow_prices[i] = constr.Pi
    print(f"Blend {i}: Shadow Price = {shadow_prices[i]:.4f}, Demand = {demand[i]}")

# Compute the total expected increase in the objective if Qmin is increased by 1 for all blends
total_increase = sum(shadow_prices[i] * demand[i] for i in blend_ids)
print("Total expected cost increase:", total_increase)


Blend 0: Shadow Price = 0.0095, Demand = 112.5716742746937
Blend 1: Shadow Price = 0.0000, Demand = 354.56416450551217
Blend 2: Shadow Price = 0.0000, Demand = 225.74239243053069
Blend 3: Shadow Price = 0.0012, Demand = 303.42827646588114
Blend 4: Shadow Price = 0.0057, Demand = 463.0265895704372
Blend 5: Shadow Price = 0.0000, Demand = 199.71689165955
Blend 6: Shadow Price = 0.0000, Demand = 264.1531692142519
Blend 7: Shadow Price = 0.0000, Demand = 402.22045541721945
Blend 8: Shadow Price = 0.0000, Demand = 191.51926619664897
Blend 9: Shadow Price = 0.0000, Demand = 130.7919639315172
Blend 10: Shadow Price = 0.0000, Demand = 215.9005811655072
Blend 11: Shadow Price = 0.0036, Demand = 164.48851490160175
Blend 12: Shadow Price = 0.0023, Demand = 471.8790609370293
Blend 13: Shadow Price = 0.0021, Demand = 423.2481518257668
Blend 14: Shadow Price = 0.0000, Demand = 353.36150260416935
Blend 15: Shadow Price = 0.0000, Demand = 448.5842360750871
Blend 16: Shadow Price = 0.0000, Demand = 421

In [14]:
# Retrieve the shadow price (dual value) for the availability constraint of raw material 73
constr73 = model.getConstrByName("Availability_material_73")
print("Shadow price for raw material 73:", constr73.Pi)


Shadow price for raw material 73: 0.0


In [15]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model for the nonlinear formulation to distinguish it from the linear model.
nonlinear_model = gp.Model("EcoClean_Blending_Nonlinear")

# Decision variables: x_nl[i,j] = tons of raw material j used in blend i (new variable names)
x_nl = {}
for i in blend_ids:
    for j in material_ids:
        x_nl[i, j] = nonlinear_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")
nonlinear_model.update()

# Objective: Minimize total production cost (same as before)
nonlinear_model.setObjective(
    gp.quicksum(cost[j] * x_nl[i, j] for i in blend_ids for j in material_ids),
    GRB.MINIMIZE
)

# Demand constraints: each blend meets its demand
for i in blend_ids:
    nonlinear_model.addConstr(
        gp.quicksum(x_nl[i, j] for j in material_ids) == demand[i],
        name=f"Demand_blend_{i}"
    )

# Quality constraints: average quality within [Qmin, Qmax]
for i in blend_ids:
    nonlinear_model.addConstr(
        gp.quicksum(quality[(i, j)] * x_nl[i, j] for j in material_ids) >= Qmin[i] * demand[i],
        name=f"QualityMin_blend_{i}"
    )
    nonlinear_model.addConstr(
        gp.quicksum(quality[(i, j)] * x_nl[i, j] for j in material_ids) <= Qmax[i] * demand[i],
        name=f"QualityMax_blend_{i}"
    )

# New nonlinear (quadratic) operational constraints:
# For each blend i and raw material j, x_nl[i,j] <= pmax[j] * (sum_{k} (x_nl[i,k])^2)
for i in blend_ids:
    quad_expr = gp.QuadExpr()
    for k in material_ids:
        quad_expr.add(x_nl[i, k] * x_nl[i, k])
    for j in material_ids:
        nonlinear_model.addQConstr(
            x_nl[i, j] <= pmax[j] * quad_expr,
            name=f"New_pmax_blend_{i}_material_{j}"
        )

# Raw material availability constraints: total usage across blends
for j in material_ids:
    nonlinear_model.addConstr(
        gp.quicksum(x_nl[i, j] for i in blend_ids) <= availability[j],
        name=f"Availability_material_{j}"
    )

nonlinear_model.update()

# Optimize the nonlinear model
nonlinear_model.optimize()

# Output the optimal cost for the nonlinear model
print("Nonlinear model optimal cost:", nonlinear_model.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 250 rows, 5000 columns and 20000 nonzeros
Model fingerprint: 0x745893db
Model has 5000 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [3e-01, 7e-01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+04]
Presolve removed 50 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve added 4995 rows and 50 columns
Presolve time: 0.14s
Presolved: 10295 rows, 10051 columns, 44455 nonzeros
Presolved model has 5000 bilinear constraint(s)
Variable types: 10051 continuous, 0 integer (0 binary)

Root relaxation: objective 3.084751e+04, 1796 iterations, 0.04 seconds (0.03 work units)

    Nodes    |    Current N

In [16]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model for the quadratic objective to distinguish it from the linear model.
quad_model = gp.Model("EcoClean_Blending_QuadObjective")

# Decision variables for the quadratic model
x_quad = {}
for i in blend_ids:
    for j in material_ids:
        x_quad[i, j] = quad_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")
quad_model.update()

# Quadratic objective: minimize sum of squared production costs
quad_model.setObjective(
    gp.quicksum(cost[j] * x_quad[i, j] * x_quad[i, j] for i in blend_ids for j in material_ids),
    GRB.MINIMIZE
)

# Demand constraints
for i in blend_ids:
    quad_model.addConstr(gp.quicksum(x_quad[i, j] for j in material_ids) == demand[i],
                         name=f"Demand_{i}")

# Quality constraints
for i in blend_ids:
    quad_model.addConstr(gp.quicksum(quality[(i, j)] * x_quad[i, j] for j in material_ids) 
                         >= Qmin[i] * demand[i], name=f"QualityMin_{i}")
    quad_model.addConstr(gp.quicksum(quality[(i, j)] * x_quad[i, j] for j in material_ids) 
                         <= Qmax[i] * demand[i], name=f"QualityMax_{i}")

# Raw material proportion constraints
for i in blend_ids:
    for j in material_ids:
        quad_model.addConstr(x_quad[i, j] <= pmax[j] * demand[i],
                             name=f"Prop_{i}_{j}")

# Raw material availability constraints
for j in material_ids:
    quad_model.addConstr(gp.quicksum(x_quad[i, j] for i in blend_ids) <= availability[j],
                         name=f"Avail_{j}")

quad_model.update()

# Optimize the quadratic model
quad_model.optimize()
print("Quadratic objective model optimal value:", quad_model.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5250 rows, 5000 columns and 25000 nonzeros
Model fingerprint: 0x4898d61a
Model has 5000 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
Presolve removed 5081 rows and 0 columns
Presolve time: 0.02s
Presolved: 169 rows, 5050 columns, 13500 nonzeros
Presolved model has 5000 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.950e+03
 Factor NZ  : 1.030e+04 (roughly 2 MB of memory)
 Factor Ops : 7.503e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    

In [19]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data for the linear model
materials_lin = pd.read_csv('/Users/allenyang/Downloads/blending_materials.csv')
blends_lin = pd.read_csv('/Users/allenyang/Downloads/blending_blends.csv')

# Define sets using DataFrame indices
material_ids_lin = list(materials_lin.index)  # Expected: 0 to 99
blend_ids_lin = list(blends_lin.index)          # Expected: 0 to 49

# Extract parameters from materials.csv (using correct column names)
cost_lin = {j: materials_lin.loc[j, 'cost'] for j in material_ids_lin}
availability_lin = {j: materials_lin.loc[j, 'availability'] for j in material_ids_lin}
pmax_lin = {j: materials_lin.loc[j, 'p_max'] for j in material_ids_lin}

# Extract parameters from blends.csv (using lowercase names)
demand_lin = {i: blends_lin.loc[i, 'demand'] for i in blend_ids_lin}
Qmin_lin = {i: blends_lin.loc[i, 'quality_min'] for i in blend_ids_lin}
Qmax_lin = {i: blends_lin.loc[i, 'quality_max'] for i in blend_ids_lin}

# Identify and sort quality contribution columns (exclude quality_min and quality_max)
quality_cols_lin = [col for col in blends_lin.columns 
                    if col.startswith('quality_') and col not in ['quality_min', 'quality_max']]
quality_cols_lin = sorted(quality_cols_lin, key=lambda x: int(x.split('_')[1]))

# Create quality parameter: quality_lin[(i, j)] for blend i and material j
quality_lin = {(i, j): blends_lin.loc[i, quality_cols_lin[j]] 
               for i in blend_ids_lin for j in range(len(material_ids_lin))}

# Build the linear model
model_lin = gp.Model("EcoClean_Blending_Linear")
x_lin = {}
for i in blend_ids_lin:
    for j in material_ids_lin:
        x_lin[i, j] = model_lin.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")
model_lin.update()

# Objective: Minimize total production cost
model_lin.setObjective(
    gp.quicksum(cost_lin[j] * x_lin[i, j] for i in blend_ids_lin for j in material_ids_lin), 
    GRB.MINIMIZE
)

# Demand satisfaction constraints
for i in blend_ids_lin:
    model_lin.addConstr(
        gp.quicksum(x_lin[i, j] for j in material_ids_lin) == demand_lin[i],
        name=f"Demand_blend_{i}"
    )

# Quality constraints
for i in blend_ids_lin:
    model_lin.addConstr(
        gp.quicksum(quality_lin[(i, j)] * x_lin[i, j] for j in material_ids_lin) >= Qmin_lin[i] * demand_lin[i],
        name=f"QualityMin_blend_{i}"
    )
    model_lin.addConstr(
        gp.quicksum(quality_lin[(i, j)] * x_lin[i, j] for j in material_ids_lin) <= Qmax_lin[i] * demand_lin[i],
        name=f"QualityMax_blend_{i}"
    )

# Original raw material proportion constraints
for i in blend_ids_lin:
    for j in material_ids_lin:
        model_lin.addConstr(
            x_lin[i, j] <= pmax_lin[j] * demand_lin[i],
            name=f"pmax_blend_{i}_material_{j}"
        )

# Raw material availability constraints
for j in material_ids_lin:
    model_lin.addConstr(
        gp.quicksum(x_lin[i, j] for i in blend_ids_lin) <= availability_lin[j],
        name=f"Availability_material_{j}"
    )

model_lin.update()
model_lin.optimize()

print("Linear model optimal cost:", model_lin.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5250 rows, 5000 columns and 25000 nonzeros
Model fingerprint: 0xb545baba
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
Presolve removed 5081 rows and 0 columns
Presolve time: 0.02s
Presolved: 169 rows, 5050 columns, 13500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9775478e+04   2.316262e+04   0.000000e+00      0s
      93    3.0879621e+04   0.000000e+00   0.000000e+00      0s

Solved in 93 iterations and 0.03 seconds (0.01 work units)
Optimal objective  3.087962126e+04
Linear model optimal cost: 30879.621255047165


In [20]:
import gurobipy as gp
from gurobipy import GRB

# Build a new model for the nonlinear formulation
nonlinear_model = gp.Model("EcoClean_Blending_Nonlinear")

# Create decision variables for the nonlinear model using the same data sets (from previous code)
x_nl = {}
for i in blend_ids_lin:
    for j in material_ids_lin:
        x_nl[i, j] = nonlinear_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_nl_{i}_{j}")
nonlinear_model.update()

# Objective: Minimize total production cost (using cost_lin from the linear model)
nonlinear_model.setObjective(
    gp.quicksum(cost_lin[j] * x_nl[i, j] for i in blend_ids_lin for j in material_ids_lin),
    GRB.MINIMIZE
)

# Demand constraints: each blend meets its demand
for i in blend_ids_lin:
    nonlinear_model.addConstr(
        gp.quicksum(x_nl[i, j] for j in material_ids_lin) == demand_lin[i],
        name=f"Demand_blend_{i}"
    )

# Quality constraints for each blend
for i in blend_ids_lin:
    nonlinear_model.addConstr(
        gp.quicksum(quality_lin[(i, j)] * x_nl[i, j] for j in material_ids_lin) >= Qmin_lin[i] * demand_lin[i],
        name=f"QualityMin_blend_{i}"
    )
    nonlinear_model.addConstr(
        gp.quicksum(quality_lin[(i, j)] * x_nl[i, j] for j in material_ids_lin) <= Qmax_lin[i] * demand_lin[i],
        name=f"QualityMax_blend_{i}"
    )

# New nonlinear (quadratic) operational constraints:
# For each blend i and raw material j, enforce:
# x_nl[i,j] <= pmax_lin[j] * (sum_{k in materials} x_nl[i,k]^2)
for i in blend_ids_lin:
    quad_expr = gp.QuadExpr()
    for k in material_ids_lin:
        quad_expr.add(x_nl[i, k] * x_nl[i, k])
    for j in material_ids_lin:
        nonlinear_model.addQConstr(
            x_nl[i, j] <= pmax_lin[j] * quad_expr,
            name=f"New_pmax_blend_{i}_material_{j}"
        )

# Raw material availability constraints (same as linear model)
for j in material_ids_lin:
    nonlinear_model.addConstr(
        gp.quicksum(x_nl[i, j] for i in blend_ids_lin) <= availability_lin[j],
        name=f"Availability_material_{j}"
    )

nonlinear_model.update()
nonlinear_model.optimize()

print("Nonlinear model optimal cost:", nonlinear_model.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 250 rows, 5000 columns and 20000 nonzeros
Model fingerprint: 0x745893db
Model has 5000 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [3e-01, 7e-01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+04]
Presolve removed 50 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve added 4995 rows and 50 columns
Presolve time: 0.14s
Presolved: 10295 rows, 10051 columns, 44455 nonzeros
Presolved model has 5000 bilinear constraint(s)
Variable types: 10051 continuous, 0 integer (0 binary)

Root relaxation: objective 3.084751e+04, 1796 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current N

### g

In [21]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data for the squared model
materials_sq = pd.read_csv('/Users/allenyang/Downloads/blending_materials.csv')
blends_sq = pd.read_csv('/Users/allenyang/Downloads/blending_blends.csv')

# Define sets using DataFrame indices
material_ids_sq = list(materials_sq.index)  # Expected: 0 to 99
blend_ids_sq = list(blends_sq.index)          # Expected: 0 to 49

# Extract parameters (using correct column names)
cost_sq = {j: materials_sq.loc[j, 'cost'] for j in material_ids_sq}
availability_sq = {j: materials_sq.loc[j, 'availability'] for j in material_ids_sq}
pmax_sq = {j: materials_sq.loc[j, 'p_max'] for j in material_ids_sq}

demand_sq = {i: blends_sq.loc[i, 'demand'] for i in blend_ids_sq}
Qmin_sq = {i: blends_sq.loc[i, 'quality_min'] for i in blend_ids_sq}
Qmax_sq = {i: blends_sq.loc[i, 'quality_max'] for i in blend_ids_sq}

# Identify and sort quality contribution columns (exclude quality_min and quality_max)
quality_cols_sq = [col for col in blends_sq.columns 
                   if col.startswith('quality_') and col not in ['quality_min', 'quality_max']]
quality_cols_sq = sorted(quality_cols_sq, key=lambda x: int(x.split('_')[1]))

# Create quality parameter: quality_sq[(i, j)] for blend i and material j
quality_sq = {(i, j): blends_sq.loc[i, quality_cols_sq[j]] 
              for i in blend_ids_sq for j in range(len(material_ids_sq))}

# Build the squared objective model
model_sq = gp.Model("EcoClean_Blending_Squared_Objective")

# Decision variables for the squared model
x_sq = {}
for i in blend_ids_sq:
    for j in material_ids_sq:
        x_sq[i, j] = model_sq.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_sq_{i}_{j}")
model_sq.update()

# Set quadratic objective: minimize sum_{i,j} (c_j * x_sq[i,j])^2 = sum_{i,j} c_j^2 * (x_sq[i,j])^2
model_sq.setObjective(
    gp.quicksum((cost_sq[j]**2) * x_sq[i, j] * x_sq[i, j] for i in blend_ids_sq for j in material_ids_sq),
    GRB.MINIMIZE
)

# Add the same constraints as in the linear model:

# 1. Demand satisfaction constraints for each blend
for i in blend_ids_sq:
    model_sq.addConstr(
        gp.quicksum(x_sq[i, j] for j in material_ids_sq) == demand_sq[i],
        name=f"Demand_blend_{i}"
    )

# 2. Quality constraints for each blend
for i in blend_ids_sq:
    model_sq.addConstr(
        gp.quicksum(quality_sq[(i, j)] * x_sq[i, j] for j in material_ids_sq) >= Qmin_sq[i] * demand_sq[i],
        name=f"QualityMin_blend_{i}"
    )
    model_sq.addConstr(
        gp.quicksum(quality_sq[(i, j)] * x_sq[i, j] for j in material_ids_sq) <= Qmax_sq[i] * demand_sq[i],
        name=f"QualityMax_blend_{i}"
    )

# 3. Raw material proportion constraints
for i in blend_ids_sq:
    for j in material_ids_sq:
        model_sq.addConstr(
            x_sq[i, j] <= pmax_sq[j] * demand_sq[i],
            name=f"pmax_blend_{i}_material_{j}"
        )

# 4. Raw material availability constraints
for j in material_ids_sq:
    model_sq.addConstr(
        gp.quicksum(x_sq[i, j] for i in blend_ids_sq) <= availability_sq[j],
        name=f"Availability_material_{j}"
    )

model_sq.update()
model_sq.optimize()

print("Squared objective model optimal value:", model_sq.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5250 rows, 5000 columns and 25000 nonzeros
Model fingerprint: 0x2674cd76
Model has 5000 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
Presolve removed 5081 rows and 0 columns
Presolve time: 0.01s
Presolved: 169 rows, 5050 columns, 13500 nonzeros
Presolved model has 5000 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.950e+03
 Factor NZ  : 1.030e+04 (roughly 2 MB of memory)
 Factor Ops : 7.503e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    